# OpenAI and Custom Tool with LangChain to Extract Director Information

This notebook contains code for creating a custom:

Tool that "reads" annual reports
Agent that uses this tool to:
\1. Identify the list of board members
\2. For each board member, extract their biography
\3. For each biography, save key information in a Dataframe

In our example, the PDF is the 2021 proxy statement for Pfizer. 

# Step 1. Download dependencies

In [ ]:
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken
%pip install pypdf
import os
import openai

In [1]:
from langchain.document_loaders import PyPDFLoader

# Load Pfizer's proxy statement. This may take 1-2 minutes since the PDF is 110 pages
proxy_statement_pdf = "https://s28.q4cdn.com/781576035/files/doc_financials/2022/sr/Proxy-Statement-2023.pdf"
# Create your PDF loader
loader = PyPDFLoader(proxy_statement_pdf)
# Load the PDF document
documents = loader.load()  

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
# Chunk the annual_report
docs = text_splitter.split_documents(documents)

# Step 2. Save the annual report
Using ChromaDB, save the annual report to a vector database. 

This will allow your custom Agent and Tool to later retrieve (use) the annual report for question-answering.

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(docs, embeddings)

Using embedded DuckDB without persistence: data will be transient


# Step 3. Create your custom Chain
This Chain will be used by your custom Tool (defined next) to answer questions
about the annual report that you previously loaded.

In [4]:
from langchain.chains.base import Chain
from typing import Dict, List

class AnnualReportChain(Chain):
    chain: Chain

    @property
    def input_keys(self) -> List[str]:
        return list(self.chain.input_keys)

    @property
    def output_keys(self) -> List[str]:
        return ['output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        # Queries the database to get the relevant documents for a given query
        query = inputs.get("input_documents", "")
        docs = vectorstore.similarity_search(query, include_metadata=True)
        output = chain.run(input_documents=docs, question=query)
        return {'output': output}

# Step 4. Create your custom Tool
This tool will use the Chain that you just created, under the hood.

In [5]:
from langchain.agents import Tool
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# Initialize your custom Chain
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo")
chain = load_qa_chain(llm)
proxy_statement_chain = AnnualReportChain(chain=chain)

# Initialize your custom Tool
proxy_statement_tool = Tool(
    name="Proxy Statement",
    func=proxy_statement_chain.run,
    description="""
    Useful for when you need to answer questions about a company's board of directors. 
    This tool can help you extract data points like the names of director nominees, their past experience,
    skills and current positions held.
    """
)

/Users/nick-mbp/opt/anaconda3/lib/python3.9/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/nick-mbp/opt/anaconda3/lib/python3.9/site-packages/langchain/llms/openai.py:676: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Step 5. Create your custom Agent
This Agent uses your custom tool(s) to get things done.

For our example, the Agent is given 1 tool (`proxy_statement_tool` from above) and answers questions about annual reports!

The code here is heavily borrowed from [this wonderful GitHub repository](https://github.com/mpaepper/llm_agents), which is created by [Marc Päpper](https://twitter.com/mpaepper).

Marc wrote an [excellent blog post](https://www.paepper.com/blog/posts/intelligent-agents-guided-by-llms/) that explains how Agents work.


In [6]:
import re

from pydantic import BaseModel
from typing import Tuple

class Agent(BaseModel):
    # The large language model that the Agent will use to decide the action to take
    llm: BaseModel
    # The prompt that the language model will use and append previous responses to
    prompt: str
    # The list of tools that the Agent can use
    tools: List[Tool]
    # Adjust this so that the Agent does not loop infinitely
    max_loops: int = 20
    # The stop pattern is used, so the LLM does not hallucinate until the end
    stop_pattern: List[str]

    @property
    def tool_by_names(self) -> Dict[str, Tool]:
        return {tool.name: tool for tool in self.tools}

    def run(self, question: str):
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        previous_responses = []
        num_loops = 0
        while num_loops < self.max_loops:
            num_loops += 1
            curr_prompt = prompt.format(previous_responses=('\n'.join(previous_responses)))
            output, tool, tool_input = self._get_next_action(curr_prompt)
            if tool == 'Final Answer':
                return tool_input
            tool_result = name_to_tool_map[tool].run(tool_input)
            output += f"\n{OBSERVATION_TOKEN} {tool_result}\n{THOUGHT_TOKEN}"
            print(output)
            previous_responses.append(output)

    def _get_next_action(self, prompt: str) -> Tuple[str, str, str]:
        # Use the LLM to generate the Agent's next action
        result = self.llm.generate([prompt], stop=self.stop_pattern)

        # List of the things generated. This is List[List[]] because each input could have multiple generations.
        generations = result.generations

        # Grab the first text generation, as this will likely be the best result
        output = generations[0][0].text

        # Parse the result
        tool, tool_input = self._get_tool_and_input(output)
        return output, tool, tool_input

    def _get_tool_and_input(self, generated: str) -> Tuple[str, str]:
        if FINAL_ANSWER_TOKEN in generated:
            return "Final Answer", generated.split(FINAL_ANSWER_TOKEN)[-1].strip()

        regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, generated, re.DOTALL)
        if not match:
            raise ValueError(f"Output of LLM is not parsable for next tool use: `{generated}`")
        tool = match.group(1).strip()
        tool_input = match.group(2)
        return tool, tool_input.strip(" ").strip('"')

# Step 6. Create your Prompt template
This prompt will be fed into the Agent's large language model (LLM).  

As it "reasons" and answers your query, the Agent will update this prompt by appending the previous response (context) to the prompt to maintain context of its overall "chain of thought".

In [7]:
FINAL_ANSWER_TOKEN = "Final Answer:"
OBSERVATION_TOKEN = "Observation:"
THOUGHT_TOKEN = "Thought:"
PROMPT_TEMPLATE = """Answer the question as best as you can using the following tools: 

{tool_description}

Use the following format:

Question: the input question you must answer
Thought: comment on what you want to do next
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action, including extracting any relevant text that answers the question
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question

Begin!

Question: {question}
Thought: {previous_responses}
"""

# Step 7. Run your custom Agent
You can update the `question` variable to ask your Agent to answer questions about the PDF that you previously loaded!


In [10]:
# The tool(s) that your Agent will use
tools = [proxy_statement_tool]

# The question that you will ask your Agent
question = "Who are the director nominees?"

# The prompt that your Agent will use and update as it is "reasoning"
prompt = PROMPT_TEMPLATE.format(
  tool_description="\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
  tool_names=", ".join([tool.name for tool in tools]),
  question=question,
  previous_responses='{previous_responses}',
)

# The LLM that your Agent will use
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo")

# Initialize your Agent
agent = Agent(
  llm=llm, 
  tools=tools, 
  prompt=prompt, 
  stop_pattern=[f'\n{OBSERVATION_TOKEN}', f'\n\t{OBSERVATION_TOKEN}'],
)

# Run the Agent!
result = agent.run(question)

print(result)

I need to find the list of director nominees for a company.
Action: Proxy Statement
Action Input: Search for the most recent proxy statement for the company.
Observation: The most recent proxy statement for Pfizer can be found on their website at https://investors.pfizer.com/financials/proxy-materials/default.aspx.
Thought:
I need to search for the list of director nominees in the proxy statement.
Action: Proxy Statement
Action Input: Open the most recent proxy statement for Pfizer.
Observation: The most recent proxy statement for Pfizer is the 2023 Proxy Statement, which is available on their website at https://investors.pfizer.com/Investors/Corporate-Governance/Contact-Our-Directors/default.aspx.
Thought:
I need to search for the section of the proxy statement that lists the director nominees.
Action: Proxy Statement
Action Input: Navigate to the section titled "Election of Directors" in the 2023 Proxy Statement for Pfizer.
Observation: The section titled "Election of Directors" can 